In [ ]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
import os


batch_size = 32
num_classes = 10
epochs = 100
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

In [ ]:
# The data, split between train and test sets:

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [ ]:
from fedml import Alliance, AllianceMember,KerasSequentialBaseLearner, FedAveragingClassifier
from read_data import read_training_data
import numpy as np

alliance = Alliance()
M=10
def split_data(x,y,fractions):
    s = len(x)
    new_order = np.random.permutation(s)
    x = x[new_order]
    y = y[new_order]
    if type(fractions) is int:
        fraction_splits = np.int32(np.linspace(0,s,fractions+1))
        nr = fractions
    elif type(fractions)==np.ndarray or type(fractions)==list:
        parts = np.array(fractions)*s/np.sum(fractions)
        fraction_splits = np.int32([np.sum(parts[:i]) for i in range(len(parts)+1)])
        nr = len(parts)
        
    x_list = []
    y_list = []
    for i in range(nr):
        x_list.append(x[fraction_splits[i]:fraction_splits[i+1]]) 
        y_list.append(y[fraction_splits[i]:fraction_splits[i+1]])

    return x_list,y_list
F = [10,10,8,8,6,6,4,4,2,2,1,1]
M = len(F)
x_train_splits, y_train_splits = split_data(x_train,y_train,fractions=F)
x_test_splits, y_test_splits = split_data(x_test,y_test,fractions=F)
print("x_train shape: ", np.array(x_train).shape)
tot_len = 0
for split in x_train_splits:
    print("x_train_splits shape: ", split.shape)
    tot_len += len(split)
print("tot len: ", tot_len)
alliance.set_validation_dataset(x_test,y_test)

classes = np.arange(10)
for i in range(M):
    member = AllianceMember(x_train_splits[i], y_train_splits[i], classes=classes)
    alliance.add_member(member)

In [ ]:
from cifarmodel import KerasSequentialCifar

base_learner = KerasSequentialCifar()
fedavg_model = FedAveragingClassifier(alliance=alliance,base_learner=base_learner)


In [ ]:
parameters = {"nr_global_iterations": 300, "nr_local_iterations":1, "training_steps": None, "data_augmentation": True, "model_size_averaging": True} 
training_loss, test_loss = fedavg_model.fit(parameters)

In [ ]:
from matplotlib import pylab as plt
import pickle
pickle.dump(fedavg_model.test_loss,open('test_loss_500e_12members_diffsizes.p','wb'))


plt.plot(fedavg_model.test_loss)
plt.savefig('accuracy_progress')
plt.show()
